In [1]:
import numpy as np
import os

In [11]:
root = '../orange_demo/baseline_new/'

In [12]:
paths = []
with open(os.path.join(root,'img_path.txt'), 'r') as f:
    for path in f.readlines():
        paths.append(path.strip())

camids = []
with open(os.path.join(root,'camids.txt'), 'r') as f:
    for camid in f.readlines():
        camids.append(int(camid.strip()))

pids = []
with open(os.path.join(root,'pids.txt'), 'r') as f:
    for pid in f.readlines():
        pids.append(int(pid.strip()))

num_query = 3843
num_gallery = 17242

In [13]:
dist = np.load(os.path.join(root, 'dist.npy'))

In [14]:
len(dist[0])

17242

In [15]:
def sort_rows_and_get_indices(array):
    # 获取每一行排序后的索引
    # sorted_indices = np.argsort(array, axis=1)[:, ::-1]
    sorted_indices = np.argsort(array, axis=1)


    # 对每一行进行排序
    sorted_rows = np.take_along_axis(array, sorted_indices, axis=1)

    return sorted_rows, sorted_indices

# 举例一个二维数组
example_array = np.array([[3, 1, 4, 1],
                          [5, 9, 2, 6],
                          [5, 3, 5, 8]])

# # 对每一行进行排序并获取索引
# sorted_rows, indices = sort_rows_and_get_indices(example_array)

# # 打印结果
# print("原始数组:")
# print(example_array)
# print("\n每一行排序后的数组:")
# print(sorted_rows)
# print("\n排序后元素在原始行中的索引:")
# print(indices)


In [16]:
sorted_dist, sorted_indice = sort_rows_and_get_indices(dist)

In [17]:
from tqdm import tqdm
def get_top_k_similar_pids(array, k):
    top_k_matches = []
    for q_id, g_ids in tqdm(enumerate(sorted_indice)):
        q_pid = pids[q_id]
        q_camid = camids[q_id]
        tmp_list = []
        l = 0
        for g_id in g_ids:
            if l >= k:
                break
            g_camid = camids[num_query + g_id]
            if q_camid == g_camid:
                continue
            g_pid = pids[num_query + g_id]
            tmp_list.append([paths[num_query + g_id], g_pid==q_pid])
            l += 1
        top_k_matches.append(tmp_list)
    return top_k_matches

In [18]:
top_5_matches = get_top_k_similar_pids(sorted_indice, 1)

3843it [00:00, 146388.67it/s]


In [19]:
top_5_matches[3]

[['datasets/Wildtrack_rerank/test/0119_c7_t1985.png', True]]

In [20]:
len(top_5_matches)


3843

In [21]:
my_dict = {}
for i in range(len(top_5_matches)):
    my_dict[paths[i]] = top_5_matches[i]

for i, x in enumerate(sorted_indice):
    print(x[:10])
    for item in x:
        print(item)
        break
    break
    pass

In [22]:
import json

In [23]:
with open(os.path.join(root, 'matches.json'), 'w') as f:
    json.dump(my_dict, f, indent=4, sort_keys=True)